데이터 전처리

In [1]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath("../"))

from utils import *
from MLP import train_and_evaluate
from permutation import permutation

# Load dataset
region="jeonju"
path = f"../../collect_data/filtered/kma/merged/kma_{region}_meta.csv"

# 2. CSV 파일 읽어서 데이터프레임으로 변환
korea_data = pd.read_csv(path)
china_data = concat_china_data(korea_data)

korea_data = convert_timesteps(korea_data)
print_missing_info(korea_data)

china_data = convert_timesteps(china_data)
print_missing_info(china_data)

(40185, 23)
IX    80.059724
dtype: float64
Index(['TM', 'STN', 'WD', 'WS', 'PA', 'PS', 'TA', 'TD', 'HM', 'PV', 'WW',
       'CA_TOT', 'CA_MID', 'CT', 'VS', 'TS', 'IR', 'PM10', 'IX', '지점', '위도',
       '경도', 'diff'],
      dtype='object')
(9488, 108)
IX             83.083895
VV              0.031619
WD_yanan        0.021079
WS_yanan        0.021079
TA_yanan        0.105396
TD_yanan        0.274030
HM_yanan        0.685076
PS_yanan        0.031619
PR              0.063238
VV_tongliao     0.010540
WD_tongliao     0.010540
WS_tongliao     0.010540
TA_tongliao     0.137015
TD_tongliao     0.252951
HM_tongliao     0.242411
PS_tongliao     0.010540
PR_tongliao     0.010540
VV_qingdao      0.094857
WD_qingdao      0.115936
WS_qingdao      0.115936
TA_qingdao      0.094857
TD_qingdao      0.200253
HM_qingdao      0.790472
PS_qingdao      0.094857
PR_qingdao      0.094857
VV_chifeng      0.063238
WD_chifeng      0.021079
WS_chifeng      0.063238
TA_chifeng      0.115936
TD_chifeng      0.263491


MLP-Model - 한국만

In [2]:
removed_columns=['TM', 'PM10','WW','CT','diff']

# Features (X) and target (y)
X = korea_data.drop(columns=removed_columns)

# 결측값 처리하는 세가지 방안
#X.fillna(0, inplace=True)
X = X.fillna(X.mean())
#X.fillna(X.median(), inplace=True)

y = korea_data['PM10']

# Train the model and get the necessary data
mean_mae, test_mae, trained_model, X_train_val, y_train_val, X_test, y_test = train_and_evaluate(
    X, y, test_size=0.2, n_splits=5, epochs=100, batch_size=64
)



Cross-validated Mean Absolute Error: 14.974236488342285
252/252 [==============================] - 1s 2ms/step - loss: 1820.1997 - mae: 14.5843
Test Set Mean Absolute Error: 14.584256172180176


질적 분석

In [3]:
# Evaluate the model and calculate permutation importance
mean_mae, low_importance_features_korea, features_with_scores = permutation(
    trained_model, X_train_val, y_train_val, X_test, y_test
)

Model Evaluation - Mean Absolute Error: 12473.9990234375
252/252 [==============================] - 0s 479us/step
Feature: STN, Importance: 0.0
Feature: WD, Importance: 1.1551755752898316
Feature: WS, Importance: -0.021304394193612096
Feature: PA, Importance: 210.50795466014787
Feature: PS, Importance: 171.92940123389272
Feature: TA, Importance: -2.7776281606640625
Feature: TD, Importance: 0.2243592495489793
Feature: HM, Importance: 64.12568545611157
Feature: PV, Importance: -0.9857713627166959
Feature: CA_TOT, Importance: -0.03752584714857221
Feature: CA_MID, Importance: 0.5908111295011622
Feature: VS, Importance: 424.6587011001111
Feature: TS, Importance: -14.329494751405582
Feature: IR, Importance: -0.09756641458479862
Feature: IX, Importance: -0.08495003707412252
Feature: 지점, Importance: 0.0
Feature: 위도, Importance: 0.0
Feature: 경도, Importance: 0.0

Top 10 Important Features (Sorted Descending by Importance Score):
Feature: VS, Importance: 424.6587011001111
Feature: PA, Importance:

MLP model - 중국 데이터 포함

In [4]:
removed_columns=['TM', 'PM10','WW','CT','STN_dalian','STN_tongliao','STN_qingdao','STN_chifeng','diff']

# Features (X) and target (y)
X = china_data.drop(columns=removed_columns)

# 결측값 처리하는 세가지 방안
#X.fillna(0, inplace=True)
X = X.fillna(X.mean())
#X.fillna(X.median(), inplace=True)

y = china_data['PM10']

# Train the model and get the necessary data
mean_mae, test_mae, trained_model, X_train_val, y_train_val, X_test, y_test = train_and_evaluate(
    X, y, test_size=0.2, n_splits=5, epochs=100, batch_size=64
)

Cross-validated Mean Absolute Error: 16.179689788818358
60/60 [==============================] - 0s 789us/step - loss: 901.3080 - mae: 14.6797
Test Set Mean Absolute Error: 14.679672241210938


질적 분석

In [5]:
# Evaluate the model and calculate permutation importance
mean_mae, low_importance_features_china, features_with_scores = permutation(
    trained_model, X_train_val, y_train_val, X_test, y_test
)

Model Evaluation - Mean Absolute Error: 24358.76171875
60/60 [==============================] - 0s 574us/step
Feature: STN, Importance: 0.0
Feature: WD, Importance: 2.473563690728042e-05
Feature: WS, Importance: 8.335253005498089e-06
Feature: PA, Importance: -6.509988124889788e-05
Feature: PS, Importance: 2.8427315555745735e-06
Feature: TA, Importance: -2.052942400041502e-05
Feature: TD, Importance: 1.8677142361411824e-05
Feature: HM, Importance: -2.489001417416148e-06
Feature: PV, Importance: 2.135265567630995e-05
Feature: CA_TOT, Importance: -2.7269660495221616e-05
Feature: CA_MID, Importance: 4.628123970178422e-05
Feature: VS, Importance: 2.0452242461033164e-05
Feature: TS, Importance: -1.2412842261255719e-05
Feature: IR, Importance: 2.4002443024073727e-05
Feature: IX, Importance: 0.00012097695143893361
Feature: 지점, Importance: 0.0
Feature: 위도, Importance: 0.0
Feature: 경도, Importance: 0.0
Feature: STN_yanan, Importance: 0.0
Feature: IW, Importance: 0.0
Feature: IR_yanan, Importance:

## MLP 한국만
Feature Importance 음수인 feature 제거한 버전

In [6]:
print(low_importance_features_korea)

['STN', 'WS', 'TA', 'PV', 'CA_TOT', 'TS', 'IR', 'IX', '지점', '위도', '경도']


In [7]:
removed_columns=['TM', 'PM10','WW','CT','diff']
removed_columns+=low_importance_features_korea

# Features (X) and target (y)
X = korea_data.drop(columns=removed_columns)

# 결측값 처리하는 세가지 방안
#X.fillna(0, inplace=True)
X = X.fillna(X.mean())
#X.fillna(X.median(), inplace=True)

y = korea_data['PM10']

# Train the model and get the necessary data
mean_mae, test_mae, trained_model, X_train_val, y_train_val, X_test, y_test = train_and_evaluate(
    X, y, test_size=0.2, n_splits=5, epochs=100, batch_size=64
)

Cross-validated Mean Absolute Error: 15.39239730834961
252/252 [==============================] - 0s 589us/step - loss: 1828.3177 - mae: 15.0096
Test Set Mean Absolute Error: 15.009605407714844


In [8]:
# Evaluate the model and calculate permutation importance
mean_mae, low_importance_features, features_with_scores = permutation(
    trained_model, X_train_val, y_train_val, X_test, y_test
)

Model Evaluation - Mean Absolute Error: 14116.373046875
252/252 [==============================] - 0s 549us/step
Feature: WD, Importance: -0.2167295091370761
Feature: PA, Importance: 104.98422768124655
Feature: PS, Importance: 161.1794800842972
Feature: TD, Importance: 0.42368334997427154
Feature: HM, Importance: 30.74320282986573
Feature: CA_MID, Importance: 0.5240248182963114
Feature: VS, Importance: 287.96997329401785

Top 10 Important Features (Sorted Descending by Importance Score):
Feature: VS, Importance: 287.96997329401785
Feature: PS, Importance: 161.1794800842972
Feature: PA, Importance: 104.98422768124655
Feature: HM, Importance: 30.74320282986573
Feature: CA_MID, Importance: 0.5240248182963114
Feature: TD, Importance: 0.42368334997427154
Feature: WD, Importance: -0.2167295091370761


## MLP 중국포함
Feature Importance 음수인 feature 제거한 버전

In [9]:
print(low_importance_features_china)

['STN', 'PA', 'TA', 'HM', 'CA_TOT', 'TS', '지점', '위도', '경도', 'STN_yanan', 'IW', 'IR_yanan', 'IX_yanan', 'VV', 'WS_yanan', 'TD_yanan', 'HM_yanan', 'PS_yanan', 'RH', 'LON', 'LAT', 'IW_tongliao', 'IR_tongliao', 'IX_tongliao', 'PT_tongliao', 'RH_tongliao', 'LON_tongliao', 'LAT_tongliao', 'IW_qingdao', 'IR_qingdao', 'IX_qingdao', 'WS_qingdao', 'PT_qingdao', 'RH_qingdao', 'LON_qingdao', 'LAT_qingdao', 'IW_chifeng', 'IR_chifeng', 'IX_chifeng', 'RH_chifeng', 'LON_chifeng', 'LAT_chifeng', 'IW_dalian', 'IR_dalian', 'IX_dalian', 'TA_dalian', 'TD_dalian', 'PR_dalian', 'RH_dalian', 'LON_dalian', 'LAT_dalian']


In [10]:
removed_columns=['TM', 'PM10','WW','CT','STN_dalian','STN_tongliao','STN_qingdao','STN_chifeng','diff']
removed_columns+=low_importance_features_china

# Features (X) and target (y)
X = china_data.drop(columns=removed_columns)

# 결측값 처리하는 세가지 방안
#X.fillna(0, inplace=True)
X = X.fillna(X.mean())
#X.fillna(X.median(), inplace=True)

y = china_data['PM10']

# Train the model and get the necessary data
mean_mae, test_mae, trained_model, X_train_val, y_train_val, X_test, y_test = train_and_evaluate(
    X, y, test_size=0.2, n_splits=5, epochs=100, batch_size=64
)

Cross-validated Mean Absolute Error: 16.166373443603515
60/60 [==============================] - 0s 581us/step - loss: 884.6365 - mae: 13.7965
Test Set Mean Absolute Error: 13.79653549194336


In [11]:
# Evaluate the model and calculate permutation importance
mean_mae, low_importance_features, features_with_scores = permutation(
    trained_model, X_train_val, y_train_val, X_test, y_test
)

Model Evaluation - Mean Absolute Error: 13949.466796875
60/60 [==============================] - 0s 512us/step
Feature: WD, Importance: -0.030152796637776192
Feature: WS, Importance: -0.0011446222242739168
Feature: PS, Importance: 35.05412211084276
Feature: TD, Importance: 0.0056642065052074034
Feature: PV, Importance: 0.00424256078513281
Feature: CA_MID, Importance: 0.2505327835720891
Feature: VS, Importance: 37.77438378793759
Feature: IR, Importance: -0.0012701510757324285
Feature: IX, Importance: 0.00826383455969335
Feature: WD_yanan, Importance: -0.4470054547004111
Feature: TA_yanan, Importance: 5.0582322364789435e-05
Feature: PT, Importance: -0.005631100139726186
Feature: PR, Importance: 0.00010001661848946242
Feature: PM10_yanan, Importance: -2.229479683614227
Feature: VV_tongliao, Importance: 0.056256194379056976
Feature: WD_tongliao, Importance: 0.004289559969038237
Feature: WS_tongliao, Importance: -0.0011676791347781546
Feature: TA_tongliao, Importance: 0.009430791868908273
F